In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split

In [2]:
data = pd.read_csv("diamonds.txt", delimiter="\t")
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [3]:
target = data.price
objects = data.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(objects, target, test_size=0.25, random_state=1)

In [4]:
lin = LinearRegression()
forest = RandomForestRegressor(random_state=1)
lin.fit(X_train, y_train)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
           verbose=0, warm_start=False)

In [5]:
linpred = lin.predict(X_test)
forpred = forest.predict(X_test)
diffslin = map(abs, linpred - y_test)
diffsfor = map(abs, forpred - y_test)

In [6]:
import scipy.stats

In [7]:
scipy.stats.ttest_ind(diffslin, diffsfor)

Ttest_indResult(statistic=6.0723314311485392, pvalue=1.2775022933953753e-09)

In [8]:
from statsmodels.stats.weightstats import *

In [9]:
cm = CompareMeans(DescrStatsW(diffslin), DescrStatsW(diffsfor))
print "95%% confidence interval: [%f, %f]" % cm.tconfint_diff(usevar='unequal')

95% confidence interval: [58.019470, 113.327566]


In [10]:
import math

In [11]:
z = (9.57 - 9.5) / (0.4 / math.sqrt(160))
print z
f = scipy.stats.norm.cdf(z)
print f
2 * (1 - f)

2.21359436212
0.986571652246


0.026856695507523787

In [12]:
scipy.stats.ttest_rel(diffslin, diffsfor)

Ttest_relResult(statistic=12.745056775181849, pvalue=5.4286548181925487e-37)

In [13]:
DescrStatsW(np.array(diffslin) - np.array(diffsfor)).tconfint_mean()

(72.497265650111629, 98.8497702562657)

### for next quiz. New start.

In [14]:
train = [1]*10 + [0]*24
test = [1]*4 + [0]*12

scipy.stats.ttest_ind(train, test)

Ttest_indResult(statistic=0.31788776569561056, pvalue=0.75194894866727324)

In [15]:
data = pd.read_csv("banknotes.txt", delimiter="\t")
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [16]:
target = data.real
objects = data.drop('real', axis=1)
X_train, X_test, y_train, y_test = train_test_split(objects, target, test_size=0.25, random_state=1)

In [17]:
first_x_train = X_train[["X1", "X2", "X3"]]
second_x_train = X_train[["X4", "X5", "X6"]]
first_x_test = X_test[["X1", "X2", "X3"]]
second_x_test = X_test[["X4", "X5", "X6"]]

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
first = LogisticRegression(random_state=1)
first.fit(first_x_train, y_train)
pred1 = first.predict(first_x_test)

second = LogisticRegression(random_state=1)
second.fit(second_x_train, y_train)
pred2 = second.predict(second_x_test)
pred1[:20]

array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score

In [21]:
acc1 = accuracy_score(y_test, pred1)
acc2 = accuracy_score(y_test, pred2)

proportions_diff_z_test(proportions_diff_z_stat_ind(pred1, pred2))

NameError: name 'proportions_diff_z_test' is not defined

In [ ]:
p1 = 1 - acc1
p2 = 1 - acc2
n1 = len(pred1)
n2 = len(pred2)
P = float(p1*n1 + p2*n2) / (n1 + n2)

In [ ]:
#no correlation
z_stat = (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
proportions_diff_z_test(z_stat)

In [ ]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [ ]:
#with relation, but for accuracy
proportions_diff_z_test(proportions_diff_z_stat_rel(pred1, pred2))

In [ ]:
p1 = 1 - acc1
p2 = 1 - acc2
z = scipy.stats.norm.ppf(1 - 0.05 / 2.)
left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
(left_boundary, right_boundary)

In [ ]:
z = (541.4 - 525) / (100 / math.sqrt(100))
print z
f = scipy.stats.norm.cdf(z)
print f
1 - f

In [ ]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [ ]:
train = [1]*10 + [0]*24
test = [1]*4 + [0]*12

z_stat = proportions_diff_z_stat_ind(train, test)
z_stat

In [ ]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [ ]:
proportions_diff_z_test(z_stat, alternative='greater')

## another way for 4 and 5 questions.

In [ ]:
error1 = np.array(pred1 != y_test)
error2 = np.array(pred2 != y_test)
(error1[:15], error2[:15])

In [ ]:
proportions_diff_z_test(proportions_diff_z_stat_rel(error1, error2))

In [ ]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [ ]:
proportions_diff_confint_rel(error1, error2)

## Non parametric quiz.

In [22]:
limfoma = np.array([49,58,75,110,112,132,151,276,281,362])
stats.wilcoxon(limfoma - 200)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

In [23]:
from statsmodels.stats.descriptivestats import sign_test

In [24]:
sign_test(limfoma, 200)

(-2.0, 0.34374999999999989)

In [25]:
good = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
bad  = np.array([17,18,18,15,12,4,14,15,10])
stats.mannwhitneyu(good, bad, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

In [26]:
data = pd.read_csv("challenger.txt", delimiter="\t", index_col='Date')

In [27]:
data.head()

,Temperature,Incident
Date,,
Apr12.81,18.9,0
Nov12.81,21.1,1
Mar22.82,20.6,0
Nov11.82,20.0,0
Apr04.83,19.4,0


In [28]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [ ]:
import random

In [29]:
def stat_intervals(stat, alpha = 0.05):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [30]:
incident = data.Temperature[data.Incident == 1]
clear = data.Temperature[data.Incident == 0]

np.random.seed(0)
clear_means = map(np.mean, get_bootstrap_samples(clear, 1000))
incident_means = map(np.mean, get_bootstrap_samples(incident, 1000))

delta_means = map(lambda x: x[0] - x[1], zip(clear_means, incident_means))
stat_intervals(delta_means)

array([ 1.42299107,  7.93861607])

In [31]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [32]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [33]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [34]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [36]:
np.random.seed(0)
permutation_test(incident, clear, max_permutations = 10000)

0.0057